In [1]:
!ls data/han/AWS/

남이섬_2016.xlsx  남이섬_2020.xlsx  청평_2019.xlsx  화천_2018.xlsx
남이섬_2017.xlsx  청평_2016.xlsx    청평_2020.xlsx  화천_2019.xlsx
남이섬_2018.xlsx  청평_2017.xlsx    화천_2016.xlsx  화천_2020.xlsx
남이섬_2019.xlsx  청평_2018.xlsx    화천_2017.xlsx


In [2]:
!pwd

/home/cjinw/work/water-quality/rnn


In [3]:
%matplotlib widget
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt
    
from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [4]:
# for Korean in plot
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [6]:

folder = 'data/han/자동'
# folder = 'data/han/수질'
# folder = 'data/han/AWS'
# folder = 'data/nakdong/자동'

## 한강
## 자동
file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
              ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
              ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx']]

## 수질
# file_names = [['가평천3_2016.xlsx','가평천3_2017.xlsx','가평천3_2018.xlsx', '가평천3_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천댐1_2016.xlsx','춘천댐1_2017.xlsx','춘천댐1_2018.xlsx', '춘천댐1_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['의암_2016.xlsx','의암_2017.xlsx','의암_2018.xlsx', '의암_2019.xlsx'],
#               ['춘성교_2016.xlsx','춘성교_2017.xlsx','춘성교_2018.xlsx', '춘성교_2019.xlsx'],
#               ['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천_2016.xlsx','춘천_2017.xlsx','춘천_2018.xlsx', '춘천_2019.xlsx']
#              ]


## AWS
# file_names = [['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2018.xlsx'],
#               ['청평_2016.xlsx', '청평_2017.xlsx', '청평_2018.xlsx', '청평_2019.xlsx'],
#               ['화천_2016.xlsx', '화천_2017.xlsx', '화천_2018.xlsx', '화천_2019.xlsx']]
 

    
## 낙동강
#자동
# file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
#               ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
#               ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]


## 금강
#자동
# file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
#               ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
#               ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]
             

    
    
## 영산강
#자동
# file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
#               ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
#               ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]
    


In [7]:
target_col = '용존산소'
# 각각의 강 마다, 측정소 별로 column을 맞춰야함
# 0: 자동, 1: 수질, 2: AWS(ASOS), 
han = [':,2:11', ':,28:45', ':,26:29']
nakdong = [':,2:11', ':,2:12']
# gain_han = ['save/han/1', 'save/han/2', 'save/han/3']

# interpolation_option = True
interpolation_option = False
# gain_save_path = gain_han[2]

iloc_val = han[0]
iloc_val

':,2:11'

In [8]:
def make_timeseries(df, interpolate=None):
    print(df.shape)
    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col])
    df = df[df[date_col].notna()]

    year = pd.DatetimeIndex(df[date_col]).year.astype(np.int64)

    start = str(year[0]) + "-01-01 00:00"
#     start

    end = str(year[0]) + "-12-31 23:00"
#     end

    time_series = pd.date_range(start = start, end = end, freq='H')
    time_series
    time_series = pd.DataFrame(time_series)
    time_series.columns = [date_col]   
    time_series

    df = pd.concat([time_series, df], axis=0)
    df = df.drop_duplicates([date_col], keep="last")
    df = df.sort_values([date_col], axis=0)
    
    if interpolate :
        print('interpolation')
        df = df.interpolate(metod='spline', order=2, limit_direction='both')
        
    print(df.shape)
    return df

In [9]:
def make_dataframe(file_names, iloc_val, interpolate=None):
    day = 24*60*60
    year = (365.2425)*day

    df_full = []
    df = []
    
    for loc in range(len(file_names)):
    
        df_loc = []
        for y in range(len(file_names[loc])):
            path = os.path.join(folder, file_names[loc][y])
            print(file_names[loc][y])
            df_tmp = make_timeseries(pd.read_excel(path), interpolate=interpolate)
            
    #         df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
    #         print(df_tmp.head) 
            df_loc.append(df_tmp)
#             df_loc.append(pd.read_excel(path))
        df_full.append(pd.concat(df_loc))
    #     df.append(df_full[loc].iloc[:,2:11])
        df.append(eval(f"df_full[loc].iloc[{iloc_val}]"))
    #     df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
    #     df.append(df_full[loc].iloc[:, 2:11])
        date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
        timestamp_s = date_time.map(datetime.datetime.timestamp)
        df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
        df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
        df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
        df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
        df[loc] = df[loc].reset_index(drop=True)
        
#         print(df[loc].shape[0])
#         rows = df[loc].shape[0]
#         percentage = round(rows * 0.7)
#         df[loc] = df[loc].dropna(axis='columns',  thresh=percentage)
        

    
    return df

In [10]:
def normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
#     train_mean = df_all.mean()
#     train_std = df_all.std()
#     for i in range(len(file_names)):
#         df[i] = (df[i]-train_mean)/train_std
    
    train_mean = df_all.min()
    train_std = df_all.max()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_mean)/(train_std-train_mean)       
    
    
    return df_all, train_mean, train_std

In [11]:
##for normal
# df = make_dataframe(file_names, iloc_val)
## for interpolation
df = make_dataframe(file_names, iloc_val, interpolate=interpolation_option)
df_all, train_mean, train_std = normalize(df)

가평_2016.xlsx
(8784, 12)
(8784, 12)
가평_2017.xlsx
(8744, 12)
(8760, 12)
가평_2018.xlsx
(8730, 12)
(8760, 12)
가평_2019.xlsx
(8760, 12)
(8760, 12)
의암호_2016.xlsx
(8784, 11)
(8784, 11)
의암호_2017.xlsx
(8760, 11)
(8760, 11)
의암호_2018.xlsx
(8746, 11)
(8760, 11)
의암호_2019.xlsx
(8760, 11)
(8760, 11)
서상_2016.xlsx
(8784, 11)
(8784, 11)
서상_2017.xlsx
(8760, 11)
(8760, 11)
서상_2018.xlsx
(8751, 11)
(8760, 11)
서상_2019.xlsx
(8760, 11)
(8760, 11)


In [12]:
train_mean, train_std

(수온           0.300
 수소이온농도       6.200
 전기전도도       48.000
 용존산소         2.900
 탁도           0.000
 총유기탄소        0.700
 총질소          0.346
 총인           0.000
 클로로필-a       0.000
 Day sin     -1.000
 Day cos     -1.000
 Year sin    -1.000
 Year cos    -1.000
 dtype: float64,
 수온           32.400
 수소이온농도       10.000
 전기전도도       191.000
 용존산소         16.400
 탁도          290.300
 총유기탄소         7.200
 총질소           5.324
 총인            0.395
 클로로필-a      159.100
 Day sin       1.000
 Day cos       1.000
 Year sin      1.000
 Year cos      1.000
 dtype: float64)

In [13]:
df[2]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,0.127726,0.421053,0.405594,0.725926,0.002067,0.092308,NaN,NaN,0.017599,0.500000,1.000000,0.498667,0.999998
1,0.121495,0.421053,0.405594,0.725926,0.002067,0.092308,NaN,NaN,0.016970,0.629410,0.982963,0.499025,0.999999
2,0.112150,0.421053,0.412587,0.711111,0.002067,0.092308,NaN,NaN,0.016970,0.750000,0.933013,0.499384,1.000000
3,0.102804,0.394737,0.412587,0.711111,0.002067,0.092308,NaN,NaN,0.017599,0.853553,0.853553,0.499742,1.000000
4,0.096573,0.394737,0.405594,0.711111,0.002067,0.092308,NaN,NaN,0.016342,0.933013,0.750000,0.500100,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,0.115265,0.315789,0.440559,0.688889,0.001378,0.138462,0.248694,0.022785,0.086109,0.017037,0.629410,0.497133,0.999992
35060,0.099688,0.315789,0.447552,0.696296,0.001378,0.138462,0.231619,0.022785,0.082338,0.066987,0.750000,0.497491,0.999994
35061,0.087227,0.315789,0.447552,0.703704,0.001378,0.153846,0.249699,0.022785,0.079824,0.146447,0.853553,0.497850,0.999995
35062,0.077882,0.315789,0.454545,0.711111,0.001378,0.138462,0.229409,0.025316,0.079195,0.250000,0.933013,0.498208,0.999997


In [14]:
df[2]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,0.127726,0.421053,0.405594,0.725926,0.002067,0.092308,NaN,NaN,0.017599,0.500000,1.000000,0.498667,0.999998
1,0.121495,0.421053,0.405594,0.725926,0.002067,0.092308,NaN,NaN,0.016970,0.629410,0.982963,0.499025,0.999999
2,0.112150,0.421053,0.412587,0.711111,0.002067,0.092308,NaN,NaN,0.016970,0.750000,0.933013,0.499384,1.000000
3,0.102804,0.394737,0.412587,0.711111,0.002067,0.092308,NaN,NaN,0.017599,0.853553,0.853553,0.499742,1.000000
4,0.096573,0.394737,0.405594,0.711111,0.002067,0.092308,NaN,NaN,0.016342,0.933013,0.750000,0.500100,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,0.115265,0.315789,0.440559,0.688889,0.001378,0.138462,0.248694,0.022785,0.086109,0.017037,0.629410,0.497133,0.999992
35060,0.099688,0.315789,0.447552,0.696296,0.001378,0.138462,0.231619,0.022785,0.082338,0.066987,0.750000,0.497491,0.999994
35061,0.087227,0.315789,0.447552,0.703704,0.001378,0.153846,0.249699,0.022785,0.079824,0.146447,0.853553,0.497850,0.999995
35062,0.077882,0.315789,0.454545,0.711111,0.001378,0.138462,0.229409,0.025316,0.079195,0.250000,0.933013,0.498208,0.999997


In [15]:
# just for Create WidnowGenerator 
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [16]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
#                out_features = None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [17]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [18]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [19]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result
@property
def example2(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.val))
    # And cache it for next time
    self._example = result
  return result
@property
def example3(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.test))
    # And cache it for next time
    self._example = result
  return result


WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example
WindowGenerator.example2 = example2
WindowGenerator.example3 = example3

In [20]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [21]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [22]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [23]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
#         print(group)
        n = np.max(group)
#         print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [24]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [25]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 miss_data_load_dir='save',
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)

        # Define mask matrix
        if miss_pattern is None:
            print("pattern none")
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            print("load save")
            self.miss = MissData(load_dir=miss_data_load_dir)
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

# dgen = GainDataGenerator(df)

In [26]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [27]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

# 측정망별 반복 구간 

In [28]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)
# 1: 자동, 2: 수질, 

[[    0    22     3     0]
 [    3     9     1    22]
 [    4    14     2    31]
 ...
 [ 5451     9     9 19454]
 [ 5460     3     3 19463]
 [ 5463     7     1 19466]]
miss_data file saved


In [29]:
dgen = GainDataGenerator(df)

pattern none


In [30]:
train_df = df_all
val_df = df_all
test_df = df_all
df_all

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,4.5,7.5,126.0,10.8,0.6,1.4,2.398,0.006,5.7,-1.407890e-12,1.000000,-0.002666,0.999996
1,4.3,7.5,126.0,10.8,0.5,1.4,2.490,0.003,5.4,2.588190e-01,0.965926,-0.001950,0.999998
2,4.4,7.5,126.0,10.8,0.5,1.4,2.545,0.003,5.4,5.000000e-01,0.866025,-0.001233,0.999999
3,4.4,7.5,126.0,10.8,0.6,1.4,2.482,0.002,5.4,7.071068e-01,0.707107,-0.000516,1.000000
4,4.4,7.5,126.0,10.8,0.6,1.4,2.453,0.003,5.4,8.660254e-01,0.500000,0.000201,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,4.0,7.4,111.0,12.2,0.4,1.6,1.584,0.009,13.7,-9.659258e-01,0.258819,-0.005734,0.999984
35060,3.5,7.4,112.0,12.3,0.4,1.6,1.499,0.009,13.1,-8.660254e-01,0.500000,-0.005017,0.999987
35061,3.1,7.4,112.0,12.4,0.4,1.7,1.589,0.009,12.7,-7.071068e-01,0.707107,-0.004301,0.999991
35062,2.8,7.4,113.0,12.5,0.4,1.6,1.488,0.010,12.6,-5.000000e-01,0.866025,-0.003584,0.999994


In [31]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)

wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

## Graph 

In [32]:
wide_window.example[0].shape

load save


TensorShape([128, 120, 13])

In [33]:
wide_window.plot(plot_col=target_col)
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

make_dataset_gain: dg.no =  9335


In [34]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습_GAIN

In [35]:
val_performance = {}
performance = {}

In [36]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [37]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 학습**

In [38]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)

# gain.save(save_dir='save')

load save
load save
Epoch 1/2000
1/1 [==============================] - 1s 1s/step - gen_loss: 38.3216 - disc_loss: 0.7145 - rmse: 0.5420 - val_loss: 0.7721
Epoch 2/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 65.6994 - disc_loss: 0.5227 - rmse: 0.7730 - val_loss: 0.3472
Epoch 3/2000
1/1 [==============================] - 0s 102ms/step - gen_loss: 17.6518 - disc_loss: 0.4859 - rmse: 0.3373 - val_loss: 0.3032
Epoch 4/2000
1/1 [==============================] - 0s 157ms/step - gen_loss: 14.0731 - disc_loss: 0.3856 - rmse: 0.3017 - val_loss: 0.2525
Epoch 5/2000
1/1 [==============================] - 0s 153ms/step - gen_loss: 10.6412 - disc_loss: 0.3943 - rmse: 0.2578 - val_loss: 0.2187
Epoch 6/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 8.3196 - disc_loss: 0.3920 - rmse: 0.2119 - val_loss: 0.1874
Epoch 7/2000
1/1 [==============================] - 0s 128ms/step - gen_loss: 7.0480 - disc_loss: 0.3815 - rmse: 0.1898 - val_loss: 0.1696
Epoch

1/1 [==============================] - 0s 124ms/step - gen_loss: 0.5603 - disc_loss: 0.2943 - rmse: 0.0737 - val_loss: 0.0708
Epoch 60/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 0.5532 - disc_loss: 0.2934 - rmse: 0.0759 - val_loss: 0.0722
Epoch 61/2000
1/1 [==============================] - 0s 132ms/step - gen_loss: 0.5291 - disc_loss: 0.2795 - rmse: 0.0680 - val_loss: 0.0717
Epoch 62/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 0.4600 - disc_loss: 0.2801 - rmse: 0.0648 - val_loss: 0.0676
Epoch 63/2000
1/1 [==============================] - 0s 122ms/step - gen_loss: 0.5572 - disc_loss: 0.2843 - rmse: 0.0679 - val_loss: 0.0696
Epoch 64/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 0.4958 - disc_loss: 0.2793 - rmse: 0.0695 - val_loss: 0.0689
Epoch 65/2000
1/1 [==============================] - 0s 90ms/step - gen_loss: 0.5034 - disc_loss: 0.2856 - rmse: 0.0703 - val_loss: 0.0676
Epoch 66/2000
1/1 [================

1/1 [==============================] - 0s 114ms/step - gen_loss: 0.4577 - disc_loss: 0.2433 - rmse: 0.0622 - val_loss: 0.0637
Epoch 118/2000
1/1 [==============================] - 0s 142ms/step - gen_loss: 0.4619 - disc_loss: 0.2333 - rmse: 0.0633 - val_loss: 0.0638
Epoch 119/2000
1/1 [==============================] - 0s 142ms/step - gen_loss: 0.4816 - disc_loss: 0.2411 - rmse: 0.0632 - val_loss: 0.0623
Epoch 120/2000
1/1 [==============================] - 0s 155ms/step - gen_loss: 0.4626 - disc_loss: 0.2407 - rmse: 0.0588 - val_loss: 0.0620
Epoch 121/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 0.4535 - disc_loss: 0.2451 - rmse: 0.0605 - val_loss: 0.0640
Epoch 122/2000
1/1 [==============================] - 0s 106ms/step - gen_loss: 0.4902 - disc_loss: 0.2365 - rmse: 0.0642 - val_loss: 0.0609
Epoch 123/2000
1/1 [==============================] - 0s 144ms/step - gen_loss: 0.4774 - disc_loss: 0.2347 - rmse: 0.0645 - val_loss: 0.0656
Epoch 124/2000
1/1 [========

1/1 [==============================] - 0s 142ms/step - gen_loss: 0.4514 - disc_loss: 0.2052 - rmse: 0.0557 - val_loss: 0.0580
Epoch 176/2000
1/1 [==============================] - 0s 118ms/step - gen_loss: 0.4759 - disc_loss: 0.2147 - rmse: 0.0591 - val_loss: 0.0610
Epoch 177/2000
1/1 [==============================] - 0s 146ms/step - gen_loss: 0.5018 - disc_loss: 0.2150 - rmse: 0.0629 - val_loss: 0.0615
Epoch 178/2000
1/1 [==============================] - 0s 110ms/step - gen_loss: 0.4774 - disc_loss: 0.2121 - rmse: 0.0554 - val_loss: 0.0594
Epoch 179/2000
1/1 [==============================] - 0s 113ms/step - gen_loss: 0.5023 - disc_loss: 0.2058 - rmse: 0.0585 - val_loss: 0.0607
Epoch 180/2000
1/1 [==============================] - 0s 141ms/step - gen_loss: 0.4523 - disc_loss: 0.2120 - rmse: 0.0587 - val_loss: 0.0549
Epoch 181/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 0.4807 - disc_loss: 0.2104 - rmse: 0.0606 - val_loss: 0.0587
Epoch 182/2000
1/1 [========

1/1 [==============================] - 0s 118ms/step - gen_loss: 0.4616 - disc_loss: 0.1963 - rmse: 0.0507 - val_loss: 0.0520
Epoch 234/2000
1/1 [==============================] - 0s 150ms/step - gen_loss: 0.4422 - disc_loss: 0.1770 - rmse: 0.0506 - val_loss: 0.0513
Epoch 235/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 0.4059 - disc_loss: 0.1848 - rmse: 0.0507 - val_loss: 0.0529
Epoch 236/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 0.4567 - disc_loss: 0.1802 - rmse: 0.0528 - val_loss: 0.0569
Epoch 237/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 0.4475 - disc_loss: 0.1816 - rmse: 0.0510 - val_loss: 0.0516
Epoch 238/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 0.4543 - disc_loss: 0.1831 - rmse: 0.0524 - val_loss: 0.0540
Epoch 239/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 0.5041 - disc_loss: 0.1808 - rmse: 0.0552 - val_loss: 0.0583
Epoch 240/2000
1/1 [========

...

1/1 [==============================] - 0s 137ms/step - gen_loss: 0.5335 - disc_loss: 0.1198 - rmse: 0.0294 - val_loss: 0.0303
Epoch 1561/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 0.5966 - disc_loss: 0.1208 - rmse: 0.0299 - val_loss: 0.0322
Epoch 1562/2000
1/1 [==============================] - 0s 116ms/step - gen_loss: 0.5859 - disc_loss: 0.1170 - rmse: 0.0313 - val_loss: 0.0316
Epoch 1563/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 0.5309 - disc_loss: 0.1229 - rmse: 0.0291 - val_loss: 0.0294
Epoch 1564/2000
1/1 [==============================] - 0s 150ms/step - gen_loss: 0.5636 - disc_loss: 0.1213 - rmse: 0.0305 - val_loss: 0.0299
Epoch 1565/2000
1/1 [==============================] - 0s 120ms/step - gen_loss: 0.5542 - disc_loss: 0.1193 - rmse: 0.0302 - val_loss: 0.0329
Epoch 1566/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 0.4905 - disc_loss: 0.1220 - rmse: 0.0310 - val_loss: 0.0322
Epoch 1567/2000
1/1 [=

1/1 [==============================] - 0s 107ms/step - gen_loss: 0.5073 - disc_loss: 0.1190 - rmse: 0.0311 - val_loss: 0.0348
Epoch 1618/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 0.4789 - disc_loss: 0.1163 - rmse: 0.0314 - val_loss: 0.0349
Epoch 1619/2000
1/1 [==============================] - 0s 142ms/step - gen_loss: 0.5467 - disc_loss: 0.1137 - rmse: 0.0305 - val_loss: 0.0291
Epoch 1620/2000
1/1 [==============================] - 0s 145ms/step - gen_loss: 0.5155 - disc_loss: 0.1216 - rmse: 0.0319 - val_loss: 0.0369
Epoch 1621/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 0.5225 - disc_loss: 0.1167 - rmse: 0.0339 - val_loss: 0.0334
Epoch 1622/2000
1/1 [==============================] - 0s 147ms/step - gen_loss: 0.5696 - disc_loss: 0.1156 - rmse: 0.0291 - val_loss: 0.0329
Epoch 1623/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 0.4926 - disc_loss: 0.1202 - rmse: 0.0322 - val_loss: 0.0329
Epoch 1624/2000
1/1 [=

1/1 [==============================] - 0s 139ms/step - gen_loss: 0.4978 - disc_loss: 0.1172 - rmse: 0.0312 - val_loss: 0.0318
Epoch 1675/2000
1/1 [==============================] - 0s 113ms/step - gen_loss: 0.4833 - disc_loss: 0.1141 - rmse: 0.0318 - val_loss: 0.0276
Epoch 1676/2000
1/1 [==============================] - 0s 113ms/step - gen_loss: 0.6117 - disc_loss: 0.1145 - rmse: 0.0323 - val_loss: 0.0292
Epoch 1677/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 0.5239 - disc_loss: 0.1115 - rmse: 0.0304 - val_loss: 0.0325
Epoch 1678/2000
1/1 [==============================] - 0s 403ms/step - gen_loss: 0.5438 - disc_loss: 0.1140 - rmse: 0.0301 - val_loss: 0.0352
Epoch 1679/2000
1/1 [==============================] - 0s 133ms/step - gen_loss: 0.5652 - disc_loss: 0.1179 - rmse: 0.0354 - val_loss: 0.0343
Epoch 1680/2000
1/1 [==============================] - 0s 134ms/step - gen_loss: 0.5731 - disc_loss: 0.1112 - rmse: 0.0286 - val_loss: 0.0346
Epoch 1681/2000
1/1 [=

1/1 [==============================] - 0s 147ms/step - gen_loss: 0.6224 - disc_loss: 0.1148 - rmse: 0.0324 - val_loss: 0.0352
Epoch 1732/2000
1/1 [==============================] - 0s 136ms/step - gen_loss: 0.6325 - disc_loss: 0.1151 - rmse: 0.0299 - val_loss: 0.0330
Epoch 1733/2000
1/1 [==============================] - 0s 142ms/step - gen_loss: 0.5944 - disc_loss: 0.1124 - rmse: 0.0321 - val_loss: 0.0350
Epoch 1734/2000
1/1 [==============================] - 0s 149ms/step - gen_loss: 0.5924 - disc_loss: 0.1136 - rmse: 0.0321 - val_loss: 0.0335
Epoch 1735/2000
1/1 [==============================] - 0s 131ms/step - gen_loss: 0.5737 - disc_loss: 0.1128 - rmse: 0.0302 - val_loss: 0.0351
Epoch 1736/2000
1/1 [==============================] - 0s 155ms/step - gen_loss: 0.5799 - disc_loss: 0.1169 - rmse: 0.0343 - val_loss: 0.0336
Epoch 1737/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 0.5960 - disc_loss: 0.1147 - rmse: 0.0323 - val_loss: 0.0396
Epoch 1738/2000
1/1 [=

1/1 [==============================] - 0s 132ms/step - gen_loss: 0.6162 - disc_loss: 0.1158 - rmse: 0.0305 - val_loss: 0.0310
Epoch 1789/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 0.6061 - disc_loss: 0.1151 - rmse: 0.0301 - val_loss: 0.0326
Epoch 1790/2000
1/1 [==============================] - 0s 115ms/step - gen_loss: 0.5836 - disc_loss: 0.1128 - rmse: 0.0292 - val_loss: 0.0300
Epoch 1791/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 0.5831 - disc_loss: 0.1145 - rmse: 0.0304 - val_loss: 0.0311
Epoch 1792/2000
1/1 [==============================] - 0s 156ms/step - gen_loss: 0.5641 - disc_loss: 0.1157 - rmse: 0.0305 - val_loss: 0.0312
Epoch 1793/2000
1/1 [==============================] - 0s 163ms/step - gen_loss: 0.6102 - disc_loss: 0.1138 - rmse: 0.0321 - val_loss: 0.0319
Epoch 1794/2000
1/1 [==============================] - 0s 158ms/step - gen_loss: 0.5670 - disc_loss: 0.1163 - rmse: 0.0316 - val_loss: 0.0300
Epoch 1795/2000
1/1 [=

**학습 loss history 출력**

In [39]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**성능 측정**

In [40]:
gain.evaluate(wide_window.test.repeat(), steps=100)

load save
100/100 [==============================] - 3s 34ms/step - loss: 0.0314


0.03140364587306976

**model save**

In [41]:
gain.save(save_dir='save')

**샘플 prediction 출력**

In [42]:
wide_window.plot(gain, plot_col='총유기탄소')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
norm_df = pd.concat(df,axis=0)

In [44]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

105192
(120, 13)
120
x.shape = (105120, 13)
x_reshape.shape = (876, 120, 13)
159120
y_true.shape= (105120, 13)
159120


(1, 120, 13)

In [45]:
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

28/28 [==============================] - 0s 2ms/step - loss: 0.0000e+00


0.0

In [46]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [47]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(105120, 13) (120, 13)
(105192, 13)


In [48]:
y_pred[~np.isnan(data)] = np.nan

In [49]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Imputed Data**

In [50]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

105192
unit_shape= (120, 13)
120
n= 105120
(35040, 9)
(35040, 9)


# 예측

In [51]:
df[0].head

<bound method NDFrame.head of              수온    수소이온농도     전기전도도      용존산소        탁도     총유기탄소       총질소  \
0      0.130841  0.342105  0.545455  0.585185  0.002067  0.107692  0.412214   
1      0.124611  0.342105  0.545455  0.585185  0.001722  0.107692  0.430695   
2      0.127726  0.342105  0.545455  0.585185  0.001722  0.107692  0.441744   
3      0.127726  0.342105  0.545455  0.585185  0.002067  0.107692  0.429088   
4      0.127726  0.342105  0.545455  0.585185  0.002067  0.107692  0.423262   
...         ...       ...       ...       ...       ...       ...       ...   
35059  0.102804  0.236842  0.468531  0.718519  0.003789  0.169231  0.275613   
35060  0.099688  0.236842  0.468531  0.718519  0.003789  0.153846  0.277220   
35061  0.096573  0.236842  0.468531  0.718519  0.003789  0.153846  0.279630   
35062  0.096573  0.236842  0.468531  0.718519  0.003789  0.153846  0.276617   
35063  0.093458  0.236842  0.468531  0.718519  0.003445  0.153846  0.273001   

             총인    클로

In [52]:
ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
print(oris[0].shape, gans[0].shape)
print(oris[1].shape, gans[1].shape)
print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
# out_features = [target_col_idx]
# out_num_features = len(out_features)
# out_num_features

(35040, 13) (35040, 13)
(35040, 9) (35040, 9)
(35040, 31) (35040, 31)


In [53]:
df[0].columns

Index(['수온', '수소이온농도', '전기전도도', '용존산소', '탁도', '총유기탄소', '총질소', '총인', '클로로필-a',
       'Day sin', 'Day cos', 'Year sin', 'Year cos'],
      dtype='object')

In [131]:
MAX_EPOCHS = 150
target_col = '용존산소'
num_features = df[0].shape[1]
label_columns_indices = {name: i for i, name in enumerate(df[0])}
target_col_idx = label_columns_indices[target_col]
# target_col_idx
out_features = [target_col_idx]
# out_features = [3,5,6,7,8]
out_num_features = len(out_features)
out_num_features

1

In [132]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
#         print("out_features")
#         print(out_features)
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        
        #y = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
       
        #print(x.shape)
        #print(y.shape)
        
        
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2].to_numpy()
            
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features].to_numpy()
            #Y = self.data[idx1:idx2]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            #print("x.shape=", x.shape)
            #print('X.shape=', X.shape)
            #print(type(x), type(X))
            x = np.append(x, [X], axis = 0)
            
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**WaterDataGenerator 테스트**

In [133]:
OUT_STEPS = 24*3

In [134]:
# OUT_STEPS = 24*3
# wdg = WaterDataGenerator(train_df,
#                          batch_size=128,
#                          input_width = 24*7,
#                          label_width = OUT_STEPS,
#                          shift = OUT_STEPS,
#                          out_num_features = out_num_features,
# #                          out_features = out_features
#                         )

In [135]:
# train_df

In [136]:
# it = iter(wdg)

In [137]:
# x,y = next(it)
# x.shape, y.shape

In [138]:
def make_dataset_water(self, data):
#   print("make_dataset_water")
#   print(out_features)
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

In [139]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

**1일단위로 변환**

In [140]:
def hour_to_day_mean(array):
    
    time = 24
    return tf.reduce_mean(tf.reshape(array, [array.shape[0]//time, time]), 1)

def plot24(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
    
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    
    input_temp = hour_to_day_mean(inputs[n, :, plot_col_index])
    input_temp = input_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.input_indices),
        input_temp,
        label='Inputs', marker='.', zorder=-10)

    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.label_indices),
        label_temp,
        label='Labels', marker='.', zorder=-10, c='#2ca02c')
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
    
      plt.plot(
          hour_to_day_mean(self.label_indices),
          predict_temp,
          label='Predictions', marker='.', zorder=-10, c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [day]')

WindowGenerator.plot24 = plot24

**예측률 계산**

In [141]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [142]:
def compa(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0, windows=None, min_max_normailze=False):
       
    if example is not None :
        inputs, labels = windows
    else :
        inputs, labels = self.example
        
        
    plot_out_col_index = self.column_indices[plot_out_col]
    
    if self.label_columns:
        label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
        label_out_col_index = plot_out_col_index
    
    if model is None:
        return
    
    print("column : " + df_all.columns[plot_col])
    
    mae = 0
    mse = 0
    rmse = 0
    mape = 0
    pred_arr = []
    label_arr = []
    
    predictions = model(inputs)
    
    if min_max_normailze :
        predictions = predictions * train_std[plot_col] + train_mean[plot_col]
        labels = labels * train_std[plot_col] + train_mean[plot_col]
        
        predictions = predictions.numpy()
        labels = labels.numpy()
        
        predictions = (predictions - predictions.min()) / (predictions.max() - predictions.min())
        labels = (labels - labels.min()) / (labels.max() - labels.min())

    for n in range(len(inputs)):
        pred_temp = hour_to_day_mean(predictions[n,:,label_out_col_index]) 
        label_temp = hour_to_day_mean(labels[n,:,label_out_col_index])
        
        pred_arr.append(pred_temp[2])
        label_arr.append(label_temp[2])
        
        
    mae = mean_absolute_error(label_arr, pred_arr)
    mse = mean_squared_error(label_arr, pred_arr)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((np.array(label_arr) - np.array(pred_arr))/np.array(label_arr)) )*100
    
    print("mae:")  
    print(mae)
    
    print("mse:")  
    print(mse)
    
    print("rmse")  
    print(rmse)
    
    print("mape")  
    print(mape)
    
    

WindowGenerator.compa = compa

In [143]:
train_df.shape

(24528, 31)

In [144]:
#multi_window.compa(elman_model, plot_col=out_features[0])

In [145]:
# target_col = '용존산소'
# num_features = df[0].shape[1]
# label_columns_indices = {name: i for i, name in enumerate(df[0])}
# target_col_idx = label_columns_indices[target_col]
# # target_col_idx
# out_features = [target_col_idx]
# # out_features = [3,5,6,7,8]
# out_num_features = len(out_features)
# out_num_features

In [146]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               )
multi_window

Total window size: 240
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167]
Label indices: [168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185
 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221
 222 223 224 225 226 227 228 229 230 231 232 

In [147]:
def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
#   model.compile(loss=tf.losses.MeanSquaredError(),
#                 optimizer=tf.optimizers.Adam(),
#                 metrics=[tf.metrics.MeanSquaredError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [148]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
#     ax.plot(history.history['mean_squared_error'], label='mse')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
#     ax.plot(history.history['val_mean_squared_error'], label='val_mse')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

In [149]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               )

In [150]:
multi_val_performance = {}
multi_performance = {}

**Elman_RNN**

In [151]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [152]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [153]:
history = compile_and_fit(elman_model, multi_window)
multi_val_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['ELMAN_RNN'])
print('test performance = ', multi_performance['ELMAN_RNN'])

Epoch 1/150
1/1 [==============================] - 1s 749ms/step - loss: 0.2824 - mean_absolute_error: 0.5160 - val_loss: 0.2868 - val_mean_absolute_error: 0.5188
Epoch 2/150
1/1 [==============================] - 0s 278ms/step - loss: 0.2676 - mean_absolute_error: 0.5031 - val_loss: 0.2403 - val_mean_absolute_error: 0.4659
Epoch 3/150
1/1 [==============================] - 0s 271ms/step - loss: 0.2147 - mean_absolute_error: 0.4469 - val_loss: 0.1563 - val_mean_absolute_error: 0.3714
Epoch 4/150
1/1 [==============================] - 0s 282ms/step - loss: 0.1416 - mean_absolute_error: 0.3550 - val_loss: 0.1004 - val_mean_absolute_error: 0.2889
Epoch 5/150
1/1 [==============================] - 0s 294ms/step - loss: 0.0967 - mean_absolute_error: 0.2866 - val_loss: 0.0475 - val_mean_absolute_error: 0.1710
Epoch 6/150
1/1 [==============================] - 0s 279ms/step - loss: 0.0466 - mean_absolute_error: 0.1870 - val_loss: 0.0312 - val_mean_absolute_error: 0.1437
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 265ms/step - loss: 0.0048 - mean_absolute_error: 0.0554 - val_loss: 0.0099 - val_mean_absolute_error: 0.0773
Epoch 52/150
1/1 [==============================] - 0s 262ms/step - loss: 0.0054 - mean_absolute_error: 0.0595 - val_loss: 0.0079 - val_mean_absolute_error: 0.0678
Epoch 53/150
1/1 [==============================] - 0s 311ms/step - loss: 0.0054 - mean_absolute_error: 0.0612 - val_loss: 0.0090 - val_mean_absolute_error: 0.0718
Epoch 54/150
1/1 [==============================] - 0s 274ms/step - loss: 0.0044 - mean_absolute_error: 0.0537 - val_loss: 0.0077 - val_mean_absolute_error: 0.0687
Epoch 55/150
1/1 [==============================] - 0s 280ms/step - loss: 0.0030 - mean_absolute_error: 0.0427 - val_loss: 0.0064 - val_mean_absolute_error: 0.0624
Epoch 56/150
1/1 [==============================] - 0s 267ms/step - loss: 0.0030 - mean_absolute_error: 0.0424 - val_loss: 0.0050 - val_mean_absolute_error: 0.0577
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 274ms/step - loss: 0.0028 - mean_absolute_error: 0.0424 - val_loss: 0.0036 - val_mean_absolute_error: 0.0445
Epoch 102/150
1/1 [==============================] - 0s 270ms/step - loss: 0.0021 - mean_absolute_error: 0.0361 - val_loss: 0.0034 - val_mean_absolute_error: 0.0447
Epoch 103/150
1/1 [==============================] - 0s 300ms/step - loss: 0.0025 - mean_absolute_error: 0.0373 - val_loss: 0.0027 - val_mean_absolute_error: 0.0392
Epoch 104/150
1/1 [==============================] - 0s 279ms/step - loss: 0.0024 - mean_absolute_error: 0.0372 - val_loss: 0.0027 - val_mean_absolute_error: 0.0396
Epoch 105/150
1/1 [==============================] - 0s 303ms/step - loss: 0.0023 - mean_absolute_error: 0.0376 - val_loss: 0.0040 - val_mean_absolute_error: 0.0452
Epoch 106/150
1/1 [==============================] - 0s 285ms/step - loss: 0.0020 - mean_absolute_error: 0.0353 - val_loss: 0.0042 - val_mean_absolute_error: 0.0488
Epoch 107/150
1/1 [=====

100/100 [==============================] - 8s 83ms/step - loss: 0.0025 - mean_absolute_error: 0.0388
val performance = [0.0026929271407425404, 0.040784433484077454]
test performance =  [0.002532016020268202, 0.038834888488054276]


In [154]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [155]:
multi_window.plot2(elman_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**GRU**

In [156]:
gru_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [157]:
history = compile_and_fit(gru_model, multi_window)
multi_val_performance['GRU'] = gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = gru_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

Epoch 1/150
1/1 [==============================] - 1s 899ms/step - loss: 0.3182 - mean_absolute_error: 0.5502 - val_loss: 0.3193 - val_mean_absolute_error: 0.5451
Epoch 2/150
1/1 [==============================] - 0s 282ms/step - loss: 0.2834 - mean_absolute_error: 0.5176 - val_loss: 0.3233 - val_mean_absolute_error: 0.5517
Epoch 3/150
1/1 [==============================] - 0s 336ms/step - loss: 0.2714 - mean_absolute_error: 0.5069 - val_loss: 0.2577 - val_mean_absolute_error: 0.4907
Epoch 4/150
1/1 [==============================] - 0s 310ms/step - loss: 0.2617 - mean_absolute_error: 0.4939 - val_loss: 0.2604 - val_mean_absolute_error: 0.4914
Epoch 5/150
1/1 [==============================] - 0s 275ms/step - loss: 0.2241 - mean_absolute_error: 0.4616 - val_loss: 0.2117 - val_mean_absolute_error: 0.4418
Epoch 6/150
1/1 [==============================] - 0s 302ms/step - loss: 0.1801 - mean_absolute_error: 0.4089 - val_loss: 0.1831 - val_mean_absolute_error: 0.4079
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 300ms/step - loss: 0.0031 - mean_absolute_error: 0.0434 - val_loss: 0.0059 - val_mean_absolute_error: 0.0595
Epoch 52/150
1/1 [==============================] - 0s 318ms/step - loss: 0.0043 - mean_absolute_error: 0.0520 - val_loss: 0.0048 - val_mean_absolute_error: 0.0529
Epoch 53/150
1/1 [==============================] - 0s 291ms/step - loss: 0.0040 - mean_absolute_error: 0.0499 - val_loss: 0.0054 - val_mean_absolute_error: 0.0581
Epoch 54/150
1/1 [==============================] - 0s 309ms/step - loss: 0.0042 - mean_absolute_error: 0.0500 - val_loss: 0.0047 - val_mean_absolute_error: 0.0538
Epoch 55/150
1/1 [==============================] - 0s 273ms/step - loss: 0.0037 - mean_absolute_error: 0.0456 - val_loss: 0.0046 - val_mean_absolute_error: 0.0548
Epoch 56/150
1/1 [==============================] - 0s 314ms/step - loss: 0.0041 - mean_absolute_error: 0.0485 - val_loss: 0.0055 - val_mean_absolute_error: 0.0583
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 277ms/step - loss: 0.0033 - mean_absolute_error: 0.0445 - val_loss: 0.0040 - val_mean_absolute_error: 0.0489
Epoch 102/150
1/1 [==============================] - 0s 312ms/step - loss: 0.0039 - mean_absolute_error: 0.0500 - val_loss: 0.0037 - val_mean_absolute_error: 0.0477
Epoch 103/150
1/1 [==============================] - 0s 291ms/step - loss: 0.0030 - mean_absolute_error: 0.0440 - val_loss: 0.0031 - val_mean_absolute_error: 0.0415
Epoch 104/150
1/1 [==============================] - 0s 283ms/step - loss: 0.0034 - mean_absolute_error: 0.0440 - val_loss: 0.0041 - val_mean_absolute_error: 0.0492
Epoch 105/150
1/1 [==============================] - 0s 261ms/step - loss: 0.0031 - mean_absolute_error: 0.0439 - val_loss: 0.0043 - val_mean_absolute_error: 0.0517
Epoch 106/150
1/1 [==============================] - 0s 320ms/step - loss: 0.0030 - mean_absolute_error: 0.0436 - val_loss: 0.0035 - val_mean_absolute_error: 0.0461
Epoch 107/150
1/1 [=====

100/100 [==============================] - 9s 93ms/step - loss: 0.0030 - mean_absolute_error: 0.0422
val performance = [0.003587054554373026, 0.045599836856126785]
test performance =  [0.0029931943863630295, 0.04222751036286354]


**LSTM**

In [158]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [159]:
history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

Epoch 1/150
1/1 [==============================] - 1s 1s/step - loss: 0.2879 - mean_absolute_error: 0.5199 - val_loss: 0.3234 - val_mean_absolute_error: 0.5506
Epoch 2/150
1/1 [==============================] - 0s 245ms/step - loss: 0.2968 - mean_absolute_error: 0.5302 - val_loss: 0.2989 - val_mean_absolute_error: 0.5268
Epoch 3/150
1/1 [==============================] - 0s 241ms/step - loss: 0.2930 - mean_absolute_error: 0.5300 - val_loss: 0.2644 - val_mean_absolute_error: 0.4936
Epoch 4/150
1/1 [==============================] - 0s 236ms/step - loss: 0.2327 - mean_absolute_error: 0.4642 - val_loss: 0.2398 - val_mean_absolute_error: 0.4696
Epoch 5/150
1/1 [==============================] - 0s 216ms/step - loss: 0.2118 - mean_absolute_error: 0.4456 - val_loss: 0.1756 - val_mean_absolute_error: 0.3998
Epoch 6/150
1/1 [==============================] - 0s 266ms/step - loss: 0.1726 - mean_absolute_error: 0.3993 - val_loss: 0.1341 - val_mean_absolute_error: 0.3372
Epoch 7/150
1/1 [========

1/1 [==============================] - 0s 248ms/step - loss: 0.0146 - mean_absolute_error: 0.1028 - val_loss: 0.0240 - val_mean_absolute_error: 0.1312
Epoch 52/150
1/1 [==============================] - 0s 250ms/step - loss: 0.0127 - mean_absolute_error: 0.0905 - val_loss: 0.0169 - val_mean_absolute_error: 0.1055
Epoch 53/150
1/1 [==============================] - 0s 241ms/step - loss: 0.0124 - mean_absolute_error: 0.0942 - val_loss: 0.0188 - val_mean_absolute_error: 0.1123
Epoch 54/150
1/1 [==============================] - 0s 233ms/step - loss: 0.0127 - mean_absolute_error: 0.0953 - val_loss: 0.0167 - val_mean_absolute_error: 0.1070
Epoch 55/150
1/1 [==============================] - 0s 211ms/step - loss: 0.0131 - mean_absolute_error: 0.0953 - val_loss: 0.0159 - val_mean_absolute_error: 0.1020
Epoch 56/150
1/1 [==============================] - 0s 252ms/step - loss: 0.0129 - mean_absolute_error: 0.0937 - val_loss: 0.0170 - val_mean_absolute_error: 0.1069
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 230ms/step - loss: 0.0033 - mean_absolute_error: 0.0458 - val_loss: 0.0053 - val_mean_absolute_error: 0.0581
Epoch 102/150
1/1 [==============================] - 0s 254ms/step - loss: 0.0043 - mean_absolute_error: 0.0498 - val_loss: 0.0066 - val_mean_absolute_error: 0.0631
Epoch 103/150
1/1 [==============================] - 0s 262ms/step - loss: 0.0051 - mean_absolute_error: 0.0556 - val_loss: 0.0107 - val_mean_absolute_error: 0.0818
Epoch 104/150
1/1 [==============================] - 0s 277ms/step - loss: 0.0054 - mean_absolute_error: 0.0582 - val_loss: 0.0079 - val_mean_absolute_error: 0.0710
Epoch 105/150
1/1 [==============================] - 0s 233ms/step - loss: 0.0040 - mean_absolute_error: 0.0509 - val_loss: 0.0048 - val_mean_absolute_error: 0.0544
Epoch 106/150
1/1 [==============================] - 0s 256ms/step - loss: 0.0044 - mean_absolute_error: 0.0535 - val_loss: 0.0041 - val_mean_absolute_error: 0.0474
Epoch 107/150
1/1 [=====

1/1 [==============================] - 0s 146ms/step - loss: 0.0033 - mean_absolute_error: 0.0429
val performance = [0.00329730287194252, 0.042912814766168594]
test performance =  [0.002934718271717429, 0.04132550209760666]


**CNN model**

In [160]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [161]:
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

Epoch 1/150
1/1 [==============================] - 0s 484ms/step - loss: 0.3259 - mean_absolute_error: 0.5588 - val_loss: 0.3131 - val_mean_absolute_error: 0.5410
Epoch 2/150
1/1 [==============================] - 0s 208ms/step - loss: 0.2791 - mean_absolute_error: 0.5148 - val_loss: 0.2075 - val_mean_absolute_error: 0.4363
Epoch 3/150
1/1 [==============================] - 0s 235ms/step - loss: 0.1961 - mean_absolute_error: 0.4244 - val_loss: 0.1036 - val_mean_absolute_error: 0.2884
Epoch 4/150
1/1 [==============================] - 0s 263ms/step - loss: 0.0741 - mean_absolute_error: 0.2461 - val_loss: 0.0167 - val_mean_absolute_error: 0.1100
Epoch 5/150
1/1 [==============================] - 0s 256ms/step - loss: 0.0145 - mean_absolute_error: 0.0991 - val_loss: 0.0675 - val_mean_absolute_error: 0.2278
Epoch 6/150
1/1 [==============================] - 0s 247ms/step - loss: 0.0887 - mean_absolute_error: 0.2699 - val_loss: 0.0741 - val_mean_absolute_error: 0.2416
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 203ms/step - loss: 0.0031 - mean_absolute_error: 0.0444 - val_loss: 0.0063 - val_mean_absolute_error: 0.0632
Epoch 52/150
1/1 [==============================] - 0s 239ms/step - loss: 0.0035 - mean_absolute_error: 0.0485 - val_loss: 0.0067 - val_mean_absolute_error: 0.0653
Epoch 53/150
1/1 [==============================] - 0s 261ms/step - loss: 0.0050 - mean_absolute_error: 0.0586 - val_loss: 0.0075 - val_mean_absolute_error: 0.0692
Epoch 54/150
1/1 [==============================] - 0s 245ms/step - loss: 0.0041 - mean_absolute_error: 0.0521 - val_loss: 0.0061 - val_mean_absolute_error: 0.0622
Epoch 55/150
1/1 [==============================] - 0s 235ms/step - loss: 0.0035 - mean_absolute_error: 0.0468 - val_loss: 0.0038 - val_mean_absolute_error: 0.0486
Epoch 56/150
1/1 [==============================] - 0s 217ms/step - loss: 0.0030 - mean_absolute_error: 0.0440 - val_loss: 0.0032 - val_mean_absolute_error: 0.0453
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 225ms/step - loss: 0.0025 - mean_absolute_error: 0.0392 - val_loss: 0.0039 - val_mean_absolute_error: 0.0481
Epoch 102/150
1/1 [==============================] - 0s 240ms/step - loss: 0.0024 - mean_absolute_error: 0.0383 - val_loss: 0.0037 - val_mean_absolute_error: 0.0464
Epoch 103/150
1/1 [==============================] - 0s 238ms/step - loss: 0.0030 - mean_absolute_error: 0.0408 - val_loss: 0.0041 - val_mean_absolute_error: 0.0507
Epoch 104/150
1/1 [==============================] - 0s 231ms/step - loss: 0.0026 - mean_absolute_error: 0.0413 - val_loss: 0.0037 - val_mean_absolute_error: 0.0470
Epoch 105/150
1/1 [==============================] - 0s 206ms/step - loss: 0.0025 - mean_absolute_error: 0.0397 - val_loss: 0.0037 - val_mean_absolute_error: 0.0480
Epoch 106/150
1/1 [==============================] - 0s 228ms/step - loss: 0.0026 - mean_absolute_error: 0.0390 - val_loss: 0.0030 - val_mean_absolute_error: 0.0429
Epoch 107/150
1/1 [=====

1/1 [==============================] - 0s 111ms/step - loss: 0.0040 - mean_absolute_error: 0.0495
val performance = [0.00399960670620203, 0.04948584735393524]
test performance =  [0.0022607636637985706, 0.03669101744890213]


In [162]:
multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

1/1 [==============================] - 0s 117ms/step - loss: 0.0034 - mean_absolute_error: 0.0450
val performance = [0.0034199696965515614, 0.045006249099969864]
test performance =  [0.0022758583072572947, 0.035491328686475754]


**performance**

In [163]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [164]:
multi_window.compa(elman_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=False)

column : 용존산소
mae:
0.03547871
mse:
0.0020232329
rmse
0.044980362
mape
7.322344928979874


In [165]:
#multi_window.compa(elman_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=True)

In [166]:
multi_window.compa(gru_model, plot_col=out_features[0], windows=multi_window.example, min_max_normailze=False)

column : 용존산소
mae:
0.03708066
mse:
0.0022452148
rmse
0.047383696
mape
7.587108016014099


In [167]:
# multi_window.compa(gru_model, plot_col=out_features[0], windows=multi_window.example, min_max_normailze=True)

In [168]:
multi_window.compa(multi_lstm_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=False)

column : 용존산소
mae:
0.036063537
mse:
0.002020012
rmse
0.044944543
mape
7.368853688240051


In [169]:
# multi_window.compa(multi_lstm_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=True)

In [170]:
multi_window.compa(multi_conv_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=False)

column : 용존산소
mae:
0.028289903
mse:
0.001279204
rmse
0.03576596
mape
5.728614702820778


In [130]:
# multi_window.compa(multi_conv_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=True)